In [21]:
#import required packages
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

#read the data
df = pd.read_csv("AirQualityUCI1.csv", parse_dates=[['Date', 'Time']], sep=';')

#check the dtypes
df.dtypes

Date_Time         object
CO(GT)           float64
PT08.S1(CO)        int64
NMHC(GT)           int64
C6H6(GT)         float64
PT08.S2(NMHC)      int64
NOx(GT)            int64
PT08.S3(NOx)       int64
NO2(GT)            int64
PT08.S4(NO2)       int64
PT08.S5(O3)        int64
T                float64
RH               float64
AH               float64
dtype: object

In [22]:

df['Date_Time'] = pd.to_datetime(df.Date_Time , format = '%d/%m/%Y %H.%M.%S')
data = df.drop(['Date_Time'], axis=1)
data.index = df.Date_Time

In [23]:
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
Date_Time,2004-03-10 18:00:00,2004-03-10 19:00:00,2004-03-10 20:00:00,2004-03-10 21:00:00,2004-03-10 22:00:00,2004-03-10 23:00:00,2004-03-11 00:00:00,2004-03-11 01:00:00,2004-03-11 02:00:00,2004-03-11 03:00:00
CO(GT),2.6,2,2.2,2.2,1.6,1.2,1.2,1,0.9,0.6
PT08.S1(CO),1360,1292,1402,1376,1272,1197,1185,1136,1094,1010
NMHC(GT),150,112,88,80,51,38,31,31,24,19
C6H6(GT),11.9,9.4,9,9.2,6.5,4.7,3.6,3.3,2.3,1.7
PT08.S2(NMHC),1046,955,939,948,836,750,690,672,609,561
NOx(GT),166,103,131,172,131,89,62,62,45,-200
PT08.S3(NOx),1056,1174,1140,1092,1205,1337,1462,1453,1579,1705
NO2(GT),113,92,114,122,116,96,77,76,60,-200
PT08.S4(NO2),1692,1559,1555,1584,1490,1393,1333,1333,1276,1235


In [24]:
#missing value treatment
cols = data.columns
for j in cols:
    for i in range(0,len(data)):
       if data[j][i] == -200:
           #print(data[j][i])
           data[j][i] = data[j][i-1]



<ipython-input-24-567f904bf57a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[j][i] = data[j][i-1]


In [25]:
#checking stationarity
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = data.drop([ 'CO(GT)'], axis=1)
johan_test_temp.head(10).T
coint_johansen(johan_test_temp,-1,1).eig

array([1.75510896e-01, 1.52389933e-01, 1.15120416e-01, 1.04126281e-01,
       9.29485509e-02, 6.89397159e-02, 5.77070988e-02, 3.43554214e-02,
       3.05980659e-02, 1.18697142e-02, 2.46766099e-03, 7.09584856e-05])

In [26]:
#creating the train and validation set
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

D:\Personal\Anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
D:\Personal\Anaconda\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)


In [36]:
import math as mt
from sklearn.metrics import mean_squared_error
#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,13):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#print(pred)
#print(valid)
#print(mt.sqrt(mean_squared_error(pred[['CO(GT)']], valid[['CO(GT)']])))
#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', mt.sqrt(mean_squared_error(pred[[i]], valid[[i]])))

rmse value for CO(GT) is :  1.408688883687523
rmse value for PT08.S1(CO) is :  205.89558284022294
rmse value for NMHC(GT) is :  6.673548711348671
rmse value for C6H6(GT) is :  7.130087248705407
rmse value for PT08.S2(NMHC) is :  277.8484437679964
rmse value for NOx(GT) is :  214.78322340908937
rmse value for PT08.S3(NOx) is :  244.95769661937507
rmse value for NO2(GT) is :  66.69695211709347
rmse value for PT08.S4(NO2) is :  490.0838893409061
rmse value for PT08.S5(O3) is :  446.51541648882966
rmse value for T is :  10.721325795592787
rmse value for RH is :  17.111676248173346
rmse value for AH is :  0.5216247245182748


In [37]:
#make final predictions
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[2.34596328e+00 1.08633212e+03 2.80762173e+02 1.24130779e+01
  1.05535947e+03 2.80882233e+02 6.59534851e+02 1.68444418e+02
  1.15918056e+03 8.50845529e+02 2.73639014e+01 1.55311062e+01
  5.15317053e-01]]


D:\Personal\Anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  warnings.warn('No frequency information was'
D:\Personal\Anaconda\lib\site-packages\statsmodels\base\wrapper.py:34: FutureWarning: y is a deprecated alias for endog, will be removed in version 0.11.0
  obj = getattr(results, attr)
